In [15]:
import pycaret

In [16]:
import interpret

In [17]:
# check version
from pycaret.utils import version
version()

'2.2.2'

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('data/processed_data/Cleaned_Data.csv')
df.head()

,Unnamed: 0,ID,Name,Sex,Age,Height,Weight,NOC,Year,Sport,Winner
0,0,1,A Dijiang,0,24,180,80,CHN,1992,Basketball,0
1,1,2,A Lamusi,0,23,170,60,CHN,2012,Judo,0
2,2,3,Gunnar Nielsen Aaby,0,24,178,73,DEN,1920,Football,0
3,3,4,Edgar Lindenau Aabye,0,34,182,95,DEN,1900,Tug-Of-War,1
4,4,5,Christine Jacoba Aaftink,1,21,185,82,NED,1988,Speed Skating,0


In [20]:
df1 = df.groupby('Sport').ID.count().reset_index()
df1 = df1.sort_values(by='ID', ascending=False)
df1 = df1.head()

popular_sports = list(df1['Sport'].values)

df2 = pd.DataFrame()

for sport in popular_sports:
    df_aux = df[df['Sport'] == sport]
    df2 = pd.concat([df2, df_aux])
df = df2
df

,Unnamed: 0,ID,Name,Sex,Age,Height,Weight,NOC,Year,Sport,Winner
11,26,8,"Cornelia ""Cor"" Aalten (-Strannood)",1,18,168,71,NED,1932,Athletics,0
27,57,18,Timo Antero Aaltonen,0,31,189,130,FIN,2000,Athletics,0
52,94,31,Evald rma (rman-),0,24,174,70,EST,1936,Athletics,0
53,95,32,Olav Augunson Aarnes,0,23,180,71,NOR,1912,Athletics,0
56,98,34,Jamale (Djamel-) Aarrass (Ahrass-),0,30,187,76,FRA,2012,Athletics,0
...,...,...,...,...,...,...,...,...,...,...,...
196524,270782,135421,Goretti Alejandra Zumaya Flores,1,19,165,65,MEX,2016,Shooting,0
196525,270783,135422,"Andreas ""Andi"" Zumbach",0,22,173,66,SUI,1992,Shooting,0
196526,270786,135422,"Andreas ""Andi"" Zumbach",0,26,173,66,SUI,1996,Shooting,0
196683,271017,135524,"Nicolaus ""Nic"" Zwetnow",0,31,181,70,NOR,1960,Shooting,0


In [21]:
df = df.drop(['Unnamed: 0', 'ID', 'Name', 'NOC', 'Year'], axis=1)
df

,Sex,Age,Height,Weight,Sport,Winner
11,1,18,168,71,Athletics,0
27,0,31,189,130,Athletics,0
52,0,24,174,70,Athletics,0
53,0,23,180,71,Athletics,0
56,0,30,187,76,Athletics,0
...,...,...,...,...,...,...
196524,1,19,165,65,Shooting,0
196525,0,22,173,66,Shooting,0
196526,0,26,173,66,Shooting,0
196683,0,31,181,70,Shooting,0


In [22]:
X = df.drop('Winner', axis=1)
y = df[['Winner']]
X.shape, y.shape

((71176, 5), (71176, 1))

> Split the Data into Train and Test: 80% Train, 20% Test 

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((56940, 5), (14236, 5))

In [24]:
X_train

,Sex,Age,Height,Weight,Sport
43834,0,25,186,70,Rowing
42617,1,27,166,60,Athletics
157194,0,23,165,59,Athletics
187091,1,18,171,63,Athletics
146747,0,23,178,70,Swimming
...,...,...,...,...,...
82269,1,20,164,53,Swimming
36312,0,28,187,88,Athletics
190356,0,24,188,85,Rowing
3853,0,23,180,65,Athletics


In [25]:
y_train

,Winner
43834,0
42617,0
157194,0
187091,0
146747,0
...,...
82269,0
36312,0
190356,0
3853,0


> Split Numerical and Catergorical:

> Initializes Setup

In [26]:
data = pd.concat([X_train, y_train], axis=1)
data.head()

,Sex,Age,Height,Weight,Sport,Winner
43834,0,25,186,70,Rowing,0
42617,1,27,166,60,Athletics,0
157194,0,23,165,59,Athletics,0
187091,1,18,171,63,Athletics,0
146747,0,23,178,70,Swimming,0


In [29]:
sports = data['Sport'].unique().tolist()

dfs = []
for s in sports:
    dfs.append(data[data['Sport'] == s])
    # display(df[df['Sport']==s])

In [32]:
sports

['Rowing', 'Athletics', 'Swimming', 'Cycling', 'Shooting']

In [30]:
from pycaret.classification import *

regs = []
for elem in dfs:
    reg = setup(data=elem, 
                target = 'Winner', 
                session_id=13, 
                log_experiment=True, 
                experiment_name='some_measurement',
                fold_shuffle=True,
                normalize = True,
                normalize_method = 'zscore'
                #shuffle=True
            )
    regs.append(reg)

,Description,Value
0,session_id,13
1,Target,Winner
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(8254, 6)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,2
8,Ordinal Features,False
9,High Cardinality Features,False


AttributeError: 'NoneType' object has no attribute 'extensions_by_name'

> Compare Baseline:

In [31]:
best_model = compare_models(fold=5, sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.6465,0.5265,0.2282,0.3011,0.2596,0.0332,0.0338,0.0140
rf,Random Forest Classifier,0.6860,0.5522,0.1823,0.3493,0.2394,0.0658,0.0713,0.1600
knn,K Neighbors Classifier,0.6761,0.5298,0.1740,0.3221,0.2256,0.0436,0.0470,0.7840
et,Extra Trees Classifier,0.6758,0.5447,0.1714,0.3189,0.2229,0.0412,0.0443,0.1580
xgboost,Extreme Gradient Boosting,0.7104,0.5654,0.1236,0.3988,0.1872,0.0672,0.0851,0.1340
lightgbm,Light Gradient Boosting Machine,0.7166,0.5659,0.0950,0.4180,0.1535,0.0572,0.0815,0.0440
catboost,CatBoost Classifier,0.7256,0.5915,0.0580,0.4793,0.1028,0.0449,0.0829,0.6920
gbc,Gradient Boosting Classifier,0.7262,0.5923,0.0274,0.4566,0.0515,0.0200,0.0510,0.0800
ada,Ada Boost Classifier,0.7268,0.5756,0.0127,0.4399,0.0246,0.0084,0.0309,0.0580
nb,Naive Bayes,0.7251,0.5498,0.0102,0.2759,0.0195,0.0027,0.0106,0.0100


> Create Model:

In [ ]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8357,0.7118,0.1607,0.3643,0.2230,0.1463,0.1617
1,0.8349,0.7253,0.1487,0.3522,0.2091,0.1336,0.1492
2,0.8319,0.6981,0.1416,0.3320,0.1986,0.1213,0.1351
3,0.8327,0.7195,0.1485,0.3412,0.2069,0.1293,0.1433
4,0.8344,0.7171,0.1724,0.3659,0.2343,0.1548,0.1686
5,0.8256,0.7034,0.1468,0.3060,0.1984,0.1139,0.1237
6,0.8299,0.6998,0.1655,0.3392,0.2225,0.1395,0.1509
7,0.8276,0.6934,0.1419,0.3097,0.1946,0.1128,0.1236
8,0.8361,0.7174,0.1590,0.3661,0.2217,0.1458,0.1617
9,0.8376,0.7198,0.1538,0.3719,0.2177,0.1441,0.1617


> Tune Hyperparameters:

In [ ]:
tuned_rf = tune_model(rf, n_iter=50, optimize='f1')

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:32:55
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: 

> Compare the Model and tuned-Model:

In [ ]:
rf

In [ ]:
tuned_rf

> Analyze Model:

In [ ]:
plot_model(tuned_rf)

In [ ]:
plot_model(tuned_rf, plot = 'error')

In [ ]:
plot_model(tuned_rf, plot = 'class_report')

In [ ]:
plot_model(tuned_rf, plot = 'feature')

In [ ]:
plot_model(tuned_lg, plot = 'feature_all')

In [ ]:
evaluate_model(tuned_rf)